# Run Quarterly

In [1]:
from database.market import Market
import pandas as pd
from tqdm import tqdm
from datetime import datetime,timezone, timedelta
import pandas_datareader as pdr
from extractor.finnhub_extractor import FinnhubExtractor
from extractor.tiingo_extractor import TiingoExtractor
from time import sleep

In [2]:
start = (datetime(2021,7,29) - timedelta(days=1)).strftime("%Y-%m-%d")
end = datetime(2021,8,2).strftime("%Y-%m-%d")
market = Market()

In [3]:
market.connect()
tickers = market.retrieve_data("sp500").sort_values("Symbol")["Symbol"]
market.close()

In [4]:
market.connect()
fails = []
for ticker in tqdm(tickers):
    try:
        if "." in ticker:
            ticker = ticker.replace(".","-")
        status = "tiingo"
#         try:
#             price = pdr.DataReader(ticker, 
#                            start=start, 
#                            end=end, 
#                            data_source='yahoo')
#             price["ticker"] = ticker
#             market.store_data("portfolio_pdr_prices",price.reset_index())
#         except Exception as e:
#             print(str(e))
        tp = TiingoExtractor(ticker,start,end).extract()
        tp["ticker"] = ticker
        market.store_data("portfolio_tiingo_prices",tp.reset_index())
        fp = FinnhubExtractor.extract(ticker,start,end)
        fp = pd.DataFrame(fp)
        fp["ticker"] = ticker
        market.store_data("portfolio_finnhub_prices",fp.reset_index())
        sleep(8)
    except Exception as e:
        msg = {"status":status,"ticker":ticker,"exception":str(e)}
        print(msg,ticker)
        fails.append(ticker)
market.close()

  6%|███▎                                                        | 28/505 [03:53<52:55,  6.66s/it]

{'status': 'tiingo', 'ticker': 'ALXN', 'exception': 'If using all scalar values, you must pass an index'} ALXN


 99%|████████████████████████████████████████████████████████▋| 502/505 [1:11:16<00:12,  4.25s/it]

Expecting value: line 1 column 1 (char 0)
{'status': 'tiingo', 'ticker': 'YUM', 'exception': "'NoneType' object does not support item assignment"} YUM
Expecting value: line 1 column 1 (char 0)
{'status': 'tiingo', 'ticker': 'ZBH', 'exception': "'NoneType' object does not support item assignment"} ZBH


100%|████████████████████████████████████████████████████████▉| 504/505 [1:11:16<00:02,  2.14s/it]

Expecting value: line 1 column 1 (char 0)
{'status': 'tiingo', 'ticker': 'ZBRA', 'exception': "'NoneType' object does not support item assignment"} ZBRA
Expecting value: line 1 column 1 (char 0)
{'status': 'tiingo', 'ticker': 'ZION', 'exception': "'NoneType' object does not support item assignment"} ZION


100%|█████████████████████████████████████████████████████████| 505/505 [1:11:16<00:00,  8.47s/it]

Expecting value: line 1 column 1 (char 0)
{'status': 'tiingo', 'ticker': 'ZTS', 'exception': "'NoneType' object does not support item assignment"} ZTS


In [5]:
tp